In [1]:
import sympy as sp
import numpy as np
import seaborn as sns
import pandas as pd

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error


sp.init_printing()

**№1.1**  
Пусть прибыль в вашей компании выражается следующей функцией, которая зависит от параметра $x$ — количества производимых товаров:  
Найдите максимально возможную прибыль, которую вы можете получить, варьируя количество произведённых товаров.

In [2]:
x = sp.Symbol('x', real=True)
f = -x**4 + 6*x**3 - 4*x**2 + 80
der = sp.diff(f, x)
extremums = sp.solve(der)
print('Функция')
display(f)
print('Производная')
display(der)
print('Точки экстремума')
display(extremums)
for solve in extremums:
    print(f'Значение функции в точке {solve} = {f.subs(x, solve)}')

Функция


Производная


Точки экстремума


Значение функции в точке 0 = 80
Значение функции в точке 1/2 = 1275/16
Значение функции в точке 4 = 144


Максимальное значение функция принимает при $x=4$, минимальное - при $x=0$.

**№1.4**
Вам необходимо оптимизировать расход бумаги в принтере, который зависит от часов работы двух ключевых сотрудников следующим образом:
$$ f(x,y) = x^2 + 2y^2 $$
Где $x$ — количество часов работы 1-ого сотрудника, $y$ — 2-ого сотрудника.

Однако вы должны оплатить этим сотрудникам ровно 20 часов работы в сумме.

Какое наименьшее количество листов бумаги вы сможете потратить в таком случае?

In [3]:
y, _lambda = sp.symbols('y, \lambda', real=True)
f = x**2 + 2*y**2
phi = x + y - 20
L = f + _lambda*phi
part_ders = sp.diff(L, x), sp.diff(L, y), sp.diff(L, _lambda)
solutions = sp.solve(part_ders, [x, y, _lambda])
minimal_consumption = f.subs(x, solutions[x]).subs(y, solutions[y])
print('Функция')
display(f)
print('Условие')
display(phi)
print('Функция Лагранжа')
display(L)
print('Экстремум')
display(solutions)
print('Минимальное количество листов, которое можно потратить')
display(round(minimal_consumption))

Функция


Условие


Функция Лагранжа


Экстремум


Минимальное количество листов, которое можно потратить


**№2.7**  
**Стохастический градиентный спуск**

In [4]:
df = sns.load_dataset('diamonds')
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)
df = pd.get_dummies(df, drop_first=True)
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])

X = df.drop(columns="price")
y = df["price"]
display(X.head(3), y.head(3))

,carat,cut_Premium,cut_Very Good,cut_Good,cut_Fair,color_E,color_F,color_G,color_H,color_I,color_J,clarity_VVS1,clarity_VVS2,clarity_VS1,clarity_VS2,clarity_SI1,clarity_SI2,clarity_I1
0,0.207014,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1,0.190620,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,0.207014,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0


0    5.789960
1    5.789960
2    5.793014
Name: price, dtype: float64

In [5]:
RANDOM_STATE = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=RANDOM_STATE)

param_grid = {
    "loss": ["squared_error", "epsilon_insensitive"],
    "penalty": ["elasticnet"],
    "alpha": np.logspace(-3, 3, 10),
    "l1_ratio": np.linspace(0, 1, 10),
    "learning_rate": ["constant"],
    "eta0": np.logspace(-4, -1, 4)
}
best_grid_sgdr = GridSearchCV(
    estimator=SGDRegressor(random_state=RANDOM_STATE),
    param_grid=param_grid,
    n_jobs=-1
).fit(X_train, y_train).best_estimator_
mse = mean_squared_error(y_test, best_grid_sgdr.predict(X_test)).round(3)
print('MSE на тестовой выборке с оптимальными параметрами:', mse)

MSE на тестовой выборке с оптимальными параметрами: 0.044


# <center>Метод Ньютона
**№3.1**  
Найдите третий корень полинома, взяв за точку старта 0.7.

In [6]:
x = sp.Symbol('x')
f = 6*x**5 - 5*x**4 - 4*x**3 + 3*x**2
start_point = 0.7
f_der = sp.diff(f, x)
current_point = start_point
for i in range(3):
    current_point -= f.subs(x, current_point) / f_der.subs(x, current_point)
    print(f'Приближенный корень уравнения на {i+1} итерации')
    display(current_point)


Приближенный корень уравнения на 1 итерации


Приближенный корень уравнения на 2 итерации


Приближенный корень уравнения на 3 итерации


***
**Пример**  
Оптимизировать функцию $f(x) = x^3 - 3x^2 - 45x + 40$

In [7]:
def der_1(x):
    return 3*x**2 - 6*x -45
def der_2(x):
    return 6*x - 6

Теперь необходимо взять какую-нибудь изначальную точку. Например, пусть это будет точка $x=42$. Также нам необходима точность — её возьмем равной $0.0001$. На каждом шаге будем переходить в следующую точку по уже упомянутой выше формуле:
$$ x^{(n+1)} = x^{(n)} - \frac{f'(x^{(n)})}{f''(x^{(n)})} $$

In [8]:
def newtons_method(f, fprime, x0, tol=0.0001):
    iter_count = 0
    x_curr = x0
    f_val = f(x_curr)
    while (abs(f_val) > tol):
        f_val = f(x_curr)
        f_prime_val = fprime(x_curr)
        x_curr = x_curr - (f_val)/(f_prime_val)
        iter_count += 1
    return x_curr

newtons_method(f=der_1, fprime=der_2, x0=50, tol=0.0001)


In [9]:
from scipy.optimize import newton
newton(func=der_1, fprime=der_2, x0=50, tol=0.0001)

***
**№3.6**  
Дана функция $f(x)=x^3-72x-220$. Найдите решение уравнения $f(x)=0$ для поиска корня в окрестностях точки $x_0=12$.

In [10]:
f = sp.sympify('x^3 - 72*x - 220')
start_point = 12
der = sp.diff(f, x)
current_point = start_point
tol = 0.001
f_value = f.subs(x, current_point)
while np.abs(f_value) > tol:
    current_point -= f.subs(x, current_point) / der.subs(x, current_point)
    f_value = f.subs(x, current_point)
print(f'Приблизительный корень уравнения с помощью метода Ньютона: {round(current_point, 3)}')
    

Приблизительный корень уравнения с помощью метода Ньютона: 9.727


**№3.7**  
Найдите положительный корень для уравнения $x^2 + 9x - 5=0$.

В качестве стартовой точки возьмите $x_0 = 2.2$.



In [11]:
f = sp.sympify('x^2 + 9*x - 5')
start_point = 2.2
der = sp.diff(f, x)
current_point = start_point
tol = 0.0001
f_value = f.subs(x, current_point)
while np.abs(f_value) > tol:
    current_point -= f.subs(x, current_point) / der.subs(x, current_point)
    f_value = f.subs(x, current_point)
print(f'Приблизительный корень уравнения с помощью метода Ньютона: {round(current_point, 2)}')

Приблизительный корень уравнения с помощью метода Ньютона: 0.52


***
**№3.9**
С помощью метода Ньютона найдите точку минимума для функции $f(x) = 8x^3 - 2x^2 - 450$.
В качестве стартовой точки возьмите $42$, точность примите за $0.0001$.

In [12]:
der_1 = lambda x: 24*x**2 - 4*x
der_2 = lambda x: 48*x - 4
solve = round(newton(func=der_1, fprime=der_2, x0=42, tol=0.0001), 3 )
print(f'Приблизительный корень уравнения с помощью метода Ньютона: {solve}')   

Приблизительный корень уравнения с помощью метода Ньютона: 0.167
